# Python Operators
This example shows how to run custom Python code using a family of DALI `PythonFunction` Operators to **prototype new augmentations** or **debug the Pipeline**. The idea behind these Operators is to provide a capability to execute Python code operating on DALI tensors' data within the pipeline execution.

Reference: https://docs.nvidia.com/deeplearning/dali/user-guide/docs/examples/custom_operations/python_operator.html


## Defining  an operation
The first kind of Operator we will try is `PythonFunction`. It wraps a regular Python function and runs it inside a DALI Pipeline. 

Let us define such a function as an example. We will name it `edit_images`.

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec, cm

from nvidia.dali import ops, types
from nvidia.dali.pipeline import Pipeline